<a href="https://colab.research.google.com/github/dangcaptkd/CS114.K21-/blob/master/sarcasm_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lấy data từ kaggle!



In [0]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"quyennguyenthinh","key":"78c6c8efa90a3136ef03e49e71fb5d9b"}'}

In [0]:
!ls -lha kaggle.json
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 72 May  8 09:45 kaggle.json


In [0]:
!kaggle datasets download -d rmisra/news-headlines-dataset-for-sarcasm-detection

  0% 0.00/3.30M [00:00<?, ?B/s]
100% 3.30M/3.30M [00:00<00:00, 111MB/s]


In [0]:
!unzip news-headlines-dataset-for-sarcasm-detection

Archive:  news-headlines-dataset-for-sarcasm-detection.zip
  inflating: Sarcasm_Headlines_Dataset.json  
  inflating: Sarcasm_Headlines_Dataset_v2.json  


# load, check data

In [0]:
import pandas as pd, numpy as np, re, time
from nltk.stem.porter import PorterStemmer

In [0]:
# Loading data from json file
data = pd.read_json('Sarcasm_Headlines_Dataset.json', lines = True)

Xem thử 5 hàng đầu của dataset 

In [32]:
data.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


Check the null value

In [0]:
print(data.isnull().any(axis = 0))

article_link    False
headline        False
is_sarcastic    False
dtype: bool


# Using linear support vector classifier, Gaussuan Naive Bayes,  Logistic Regression, Random Forest Classifier

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

**clean data**

Ở cột headline sẽ có 1 vài kí tự cần được loại bỏ, dùng regular expression để loại bỏ các kí tự này

In [0]:
data['headline'] = data['headline'].apply(lambda s : re.sub('[^a-zA-Z]', ' ', s))

Gán feature và label 

In [0]:
features = data['headline']
labels = data['is_sarcastic']

Steaming data

In [65]:
print('before steaming:', features[0])
ps = PorterStemmer()
features = features.apply(lambda x: x.split())
features = features.apply(lambda x : ' '.join([ps.stem(word) for word in x]))
print('after steaming:', features[0])

before steaming: former versace store clerk sues over secret  black code  for minority shoppers
after steaming: former versac store clerk sue over secret black code for minor shopper


What is nltk.stem.porter.PorterStemmer() using for? Watch here: https://www.nltk.org/howto/stem.html

Vector hóa các tính năng bằng cách sử dụng TF-IDF Vectorizer (chỗ này để làm gì ta??)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features = 5000)
features = list(features)
features = tv.fit_transform(features).toarray()

https://en.wikipedia.org/wiki/Tf%E2%80%93idf
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html



```
`# This is formatted as code`
```

**split data** (1/9)

In [58]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.1, random_state = 0)
print('features_train:', features_train.shape)
print('labels_train:', labels_train.shape)
print('features_test', features_test.shape)
print('labels_test:', labels_test.shape)

features_train: (24038, 5000)
labels_train: (24038,)
features_test (2671, 5000)
labels_test: (2671,)


**training and testing data**

model 1: Using linear support vector classifier

In [67]:
lsvc = LinearSVC()
# training the model
lsvc.fit(features_train, labels_train)
# getting the score of train and test data
#print('train:', lsvc.loss(features_train, labels_train))
#core = lsvc.evaluate(features_test, labels_test) 
print('train:', lsvc.score(features_train, labels_train)) 
print('test:', lsvc.score(features_test, labels_test))   

train: 0.9110574923038522
test: 0.8360164732309996


model 2: Using Gaussuan Naive Bayes

In [60]:
gnb = GaussianNB()
gnb.fit(features_train, labels_train)
print('train:', gnb.score(features_train, labels_train))  
print('test:', gnb.score(features_test, labels_test))    

train: 0.7826358266078709
test: 0.7162111568700861


model 3: Logistic Regression

In [61]:
lr = LogisticRegression()
lr.fit(features_train, labels_train)
print('train:', lr.score(features_train, labels_train))   
print('test:', lr.score(features_test, labels_test))     

train: 0.8824777435726766
test: 0.8375140396855111


model 4: Random Forest Classifier

In [62]:
rfc = RandomForestClassifier(n_estimators = 10, random_state = 0)
rfc.fit(features_train, labels_train)
print('train:', rfc.score(features_train, labels_train)) 
print('test:', rfc.score(features_test, labels_test))    

train: 0.9895998003161661
test: 0.7963309621864471


# LSTM, GRU, Conv1d model (Sequential)

In [68]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, LSTM, Bidirectional, Dropout, Conv1D, MaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.layers import GlobalMaxPool1D, GRU
from keras import optimizers

Using TensorFlow backend.


In [0]:
MAX_SEQ_LENGTH = 40
MAX_WORDS = 20000

In [0]:
tok = Tokenizer(num_words = MAX_WORDS) # keeping 10000 now for first iteration
tok.fit_on_texts(data.headline)
seqs = tok.texts_to_sequences(data.headline)

What is from keras.preprocessing.text import Tokenizer using for? Watch here: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer

In [0]:
features = pad_sequences(seqs, MAX_SEQ_LENGTH)
labels = np.asarray(data.is_sarcastic)

What's pad_sequences using for? https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer

In [74]:
print('features.shape:', features.shape)
print('lables.shape:', labels.shape)

features.shape: (26709, 40)
lables.shape: (26709,)


In [0]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.1, random_state = 0)

In [76]:
print('features_train:', features_train.shape)
print('labels_train:', labels_train.shape)
print('features_test:', features_test.shape)
print('labels_test:', labels_test.shape)

features_train: (24038, 40)
labels_train: (24038,)
features_test: (2671, 40)
labels_test: (2671,)


In [0]:
EMB_DIM = 6

In [0]:
EPOCHS = 9
BATCH_SIZE =512

**Build and Train a Fully Connected model**

In [0]:
def fcmodel():
    model = Sequential()
    model.add(Embedding(input_dim=MAX_WORDS, output_dim= EMB_DIM, input_length=MAX_SEQ_LENGTH))    
    
    # Flatten Layer
    model.add(Flatten())
    
    # FC1
    model.add(Dense(64, activation='relu'))
    
    # Output layer
    model.add(Dense(1, activation='sigmoid'))
    
    # print model summary
    #model.summary()
    
    # When using pretrained embeddings
    #model.layers[0].set_weights([embedding_matrix])
    #model.layers[0].trainable = False
              
    # Compile the model
    model.compile(optimizer = 'rmsprop',
                 loss = 'binary_crossentropy',
                 metrics = ['acc'])
    return model

fcmod = fcmodel()

In [91]:
fchist = fcmod.fit(features_train, labels_train,
                   epochs = EPOCHS,
                   batch_size = BATCH_SIZE,
                   validation_data = (features_test, labels_test))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 24038 samples, validate on 2671 samples
Epoch 1/9
24038/24038 [==============================] - 0s 14us/step - loss: 0.6647 - acc: 0.5911 - val_loss: 0.6160 - val_acc: 0.6949
Epoch 2/9
24038/24038 [==============================] - 0s 10us/step - loss: 0.5216 - acc: 0.7985 - val_loss: 0.4611 - val_acc: 0.8098
Epoch 3/9
24038/24038 [==============================] - 0s 10us/step - loss: 0.3685 - acc: 0.8628 - val_loss: 0.3751 - val_acc: 0.8405
Epoch 4/9
24038/24038 [==============================] - 0s 10us/step - loss: 0.2861 - acc: 0.8900 - val_loss: 0.3468 - val_acc: 0.8514
Epoch 5/9
24038/24038 [==============================] - 0s 9us/step - loss: 0.2391 - acc: 0.9078 - val_loss: 0.3370 - val_acc: 0.8559
Epoch 6/9
24038/24038 [==============================] - 0s 10us/step - loss: 0.2054 - acc: 0.9215 - val_loss: 0.3457 - val_acc: 0.8529
Epoch 7/9
24038/24038 [==============================] - 0s 10us/step - loss: 0.1789 - acc: 0.9339 - val_loss: 0.3395 - val_acc: 0.8592


In [92]:
scores = fcmod.evaluate(features_test, labels_test)
print('loss: {}\nacc: {}'.format(scores[0], scores[1]))

2671/2671 [==============================] - 0s 27us/step
loss: 0.3567418198445369
acc: 0.8584799766540527


**Build and Train LSTM model**

In [0]:
def lstm():
    model = Sequential()
    
    model.add(Embedding(input_dim=MAX_WORDS, output_dim=EMB_DIM, input_length=MAX_SEQ_LENGTH))
    
    model.add(Bidirectional(LSTM(16, return_sequences=True, recurrent_dropout=0.1, dropout=0.1)))
    
    model.add(Bidirectional(LSTM(32, recurrent_dropout=0.1, dropout=0.1)))
    
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile the model
    model.compile(optimizer = 'rmsprop',
                  loss = 'binary_crossentropy',
                  metrics = ['acc'])
    
    return model

lsmod = lstm()

In [94]:
lshist = lsmod.fit(features_train, labels_train,
         epochs = EPOCHS,
         batch_size = BATCH_SIZE,
         validation_data = (features_test, labels_test))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 24038 samples, validate on 2671 samples
Epoch 1/9
24038/24038 [==============================] - 14s 591us/step - loss: 0.6342 - acc: 0.6255 - val_loss: 0.4664 - val_acc: 0.7806
Epoch 2/9
24038/24038 [==============================] - 13s 523us/step - loss: 0.3902 - acc: 0.8354 - val_loss: 0.3767 - val_acc: 0.8383
Epoch 3/9
24038/24038 [==============================] - 13s 524us/step - loss: 0.3083 - acc: 0.8783 - val_loss: 0.3579 - val_acc: 0.8450
Epoch 4/9
24038/24038 [==============================] - 13s 523us/step - loss: 0.2542 - acc: 0.9035 - val_loss: 0.3470 - val_acc: 0.8469
Epoch 5/9
24038/24038 [==============================] - 13s 525us/step - loss: 0.2113 - acc: 0.9229 - val_loss: 0.3590 - val_acc: 0.8521
Epoch 6/9
24038/24038 [==============================] - 13s 525us/step - loss: 0.1816 - acc: 0.9342 - val_loss: 0.3621 - val_acc: 0.8525
Epoch 7/9
24038/24038 [==============================] - 13s 520us/step - loss: 0.1547 - acc: 0.9457 - val_loss: 0.3644 - v

**Build and Train Conv1D model**

In [0]:
def conv1d():
    
    model = Sequential()
    
    model.add(Embedding(input_dim=MAX_WORDS,output_dim=EMB_DIM, input_length=MAX_SEQ_LENGTH))
    
    model.add(Conv1D(filters=32, kernel_size=7, activation='relu'))
    
    model.add(MaxPool1D(pool_size=5))
    
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
    
    model.add(Dropout(0.1))
    
    model.add(GlobalMaxPool1D())
    
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics= ['acc'])
    
    return model

convmod = conv1d()

In [96]:
convhist = convmod.fit(features_train, labels_train,
                      epochs = EPOCHS,
                      batch_size = BATCH_SIZE,
                      validation_data = (features_test, labels_test))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 24038 samples, validate on 2671 samples
Epoch 1/9
24038/24038 [==============================] - 1s 50us/step - loss: 0.6791 - acc: 0.5594 - val_loss: 0.6606 - val_acc: 0.5601
Epoch 2/9
24038/24038 [==============================] - 1s 39us/step - loss: 0.6023 - acc: 0.6771 - val_loss: 0.5462 - val_acc: 0.7649
Epoch 3/9
24038/24038 [==============================] - 1s 38us/step - loss: 0.4647 - acc: 0.8143 - val_loss: 0.4525 - val_acc: 0.7847
Epoch 4/9
24038/24038 [==============================] - 1s 38us/step - loss: 0.3685 - acc: 0.8454 - val_loss: 0.4170 - val_acc: 0.8008
Epoch 5/9
24038/24038 [==============================] - 1s 38us/step - loss: 0.3145 - acc: 0.8638 - val_loss: 0.4061 - val_acc: 0.8057
Epoch 6/9
24038/24038 [==============================] - 1s 39us/step - loss: 0.2786 - acc: 0.8789 - val_loss: 0.4065 - val_acc: 0.8109
Epoch 7/9
24038/24038 [==============================] - 1s 39us/step - loss: 0.2517 - acc: 0.8900 - val_loss: 0.4161 - val_acc: 0.8076

**Build and Train Conv1D + GRU modelt**

In [0]:
def convgru():
    
    model = Sequential()
    
    model.add(Embedding(input_dim=MAX_WORDS,output_dim=EMB_DIM, input_length=MAX_SEQ_LENGTH))
    
    model.add(Conv1D(filters=32, kernel_size=7, activation='relu'))
    
    model.add(MaxPool1D(pool_size=5))
    
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
    
    model.add(GRU(32, dropout=0.1, recurrent_dropout=0.5))
    
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics= ['acc'])
    
    return model

convgrumod = convgru()

In [98]:
convgruhist = convgrumod.fit(features_train, labels_train, 
                           epochs = EPOCHS,
                           batch_size=BATCH_SIZE,
                           validation_data = (features_test, labels_test))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 24038 samples, validate on 2671 samples
Epoch 1/9
24038/24038 [==============================] - 2s 77us/step - loss: 0.6716 - acc: 0.5884 - val_loss: 0.6312 - val_acc: 0.7005
Epoch 2/9
24038/24038 [==============================] - 1s 57us/step - loss: 0.5237 - acc: 0.7506 - val_loss: 0.4594 - val_acc: 0.7885
Epoch 3/9
24038/24038 [==============================] - 1s 58us/step - loss: 0.3773 - acc: 0.8295 - val_loss: 0.4086 - val_acc: 0.8087
Epoch 4/9
24038/24038 [==============================] - 1s 57us/step - loss: 0.3110 - acc: 0.8638 - val_loss: 0.4025 - val_acc: 0.8169
Epoch 5/9
24038/24038 [==============================] - 1s 57us/step - loss: 0.2713 - acc: 0.8837 - val_loss: 0.4134 - val_acc: 0.8121
Epoch 6/9
24038/24038 [==============================] - 1s 58us/step - loss: 0.2447 - acc: 0.8969 - val_loss: 0.4130 - val_acc: 0.8165
Epoch 7/9
24038/24038 [==============================] - 1s 57us/step - loss: 0.2193 - acc: 0.9086 - val_loss: 0.4327 - val_acc: 0.8132